In [1]:
import dualcodec
import torchaudio
from IPython.display import Audio

/home/vansh/dualcodec-exp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = "12hz_v1"

In [3]:
# use strict=False to load the model when making changes to the model layers (which changes the names in the state dictionairy)
# This is only to load random weights and confirm that the forward pass works properly

path = "hf://vanshjjw/dualcodec-whisper-la-3-v1"

dualcodec_model = dualcodec.get_model(model_id, pretrained_model_path=path, is_checkpoint=True, strict=True) 

/home/vansh/dualcodec-exp/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 39199.10it/s]

pretrained_model_path /home/vansh/.cache/huggingface/hub/models--vanshjjw--dualcodec-whisper-la-3-v1/snapshots/8ac26c079e4bd3ff123ce06c060b9574f5d55253
Loading model from here /home/vansh/.cache/huggingface/hub/models--vanshjjw--dualcodec-whisper-la-3-v1/snapshots/8ac26c079e4bd3ff123ce06c060b9574f5d55253/model.safetensors
Model loaded


In [4]:
# If you switch from InferenceWhisper to Inference (normal one with w2v) you have to change the model semantic_repr_dim to 1024 in 
# conf/model/dualcodec_12hz_16384_4096_8vq.yaml

dualcodec_inference = dualcodec.InferenceWhisper(dualcodec_model=dualcodec_model)

Fetching 7 files: 100%|██████████| 7/7 [00:00<00:00, 72137.91it/s]


In [5]:
dualcodec_inference.semantic_cfg.semantic_model.encoder.causal_mask[0][0][1][:10]

tensor([0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf], device='cuda:0')

In [6]:
audio, sr = torchaudio.load("audio_samples/tara.wav")
audio = torchaudio.functional.resample(audio, sr, 24000)
audio = audio.reshape(1,1,-1)
audio = audio.cpu()

Audio(audio.squeeze(0), rate=24000)

In [7]:
semantic_codes, acoustic_codes = dualcodec_inference.encode(audio, n_quantizers=8)

audio_16k.shape torch.Size([1, 1, 480000])
audio_input.shape torch.Size([480000])


`sdpa` attention does not support `output_attentions=True` or `head_mask`. Please set your attention to `eager` if you want any of these features.


In [8]:
print(semantic_codes.shape)
print(acoustic_codes.shape)

torch.Size([1, 1, 375])
torch.Size([1, 7, 375])


In [9]:
out_audio = dualcodec_inference.decode(semantic_codes, acoustic_codes)
Audio(out_audio.cpu().squeeze(0), rate=24000)

### Push to Hub

In [ ]:
dualcodec_model.push_to_hub("vanshjjw/dualcodec-whisper-la-3-v1")